In [36]:
import pandas as pd
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pickle
import joblib
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, GlobalAveragePooling2D, ReLU, Dense, DepthwiseConv2D, AveragePooling2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers

In [117]:
y_train=np.load('y_train_temp.npy')
y_train.shape

(365,)

In [118]:
X_train=np.load('X_train_temp.npy')
X_train.shape

(365, 192, 753, 3)

In [119]:
y_test=np.load('y_test_temp.npy')
y_test.shape

(93,)

In [120]:
X_test=np.load('X_test_temp.npy')
X_test.shape

(93, 192, 753, 3)

In [121]:
X_train = np.array((X_train-np.min(X_train))/(np.max(X_train)-np.min(X_train)))
X_test = np.array((X_test-np.min(X_test))/(np.max(X_test)-np.min(X_test)))
X_train = X_train/np.std(X_train)
X_test = X_test/np.std(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [122]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.4, random_state=123)
print(X_train.shape,  X_val.shape, len(y_train),  len(y_val))

(219, 192, 753, 3) (146, 192, 753, 3) 219 146


<Font size = 6> <b> MobileNet Model </b></font>

In [129]:
def depth_block(x, strides):
    x = DepthwiseConv2D(3,strides=strides,padding='same',  use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x
def single_conv_block(x,filters):
    x = Conv2D(filters, 1,use_bias=False)(x)
    x= BatchNormalization()(x)
    x = ReLU()(x)
    return x
def combo_layer(x,repetition, strides):
    x = depth_block(x,strides)
    filters=32
    x = single_conv_block(x, filters)
    return x
def MobileNet(input_shape=(192, 753, 3),n_classes = 4):
    input = Input (input_shape)
    x = Conv2D(32,3,strides=(2,2),padding = 'same', use_bias=False) (input)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = combo_layer(x,64, strides=(1,1))
    #x = combo_layer(x,128,strides=(2,2))
    #x = combo_layer(x,128,strides=(1,1))
    #x = combo_layer(x,256,strides=(2,2))
    #x = combo_layer(x,256,strides=(1,1))
    #x = combo_layer(x,512,strides=(2,2))
    #for _ in range(10):
    #    x = combo_layer(x,512,strides=(1,1))
    x = combo_layer(x,1024,strides=(2,2))
    #x = combo_layer(x,1024,strides=(1,1))
    #x = GlobalAveragePooling2D()(x)
    #x = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same')(x)
    x=Flatten()(x)
    output = Dense(n_classes,activation='softmax')(x)
    model = Model(input, output)
    return model
n_classes = 4
input_shape = (192, 753, 3)
model = MobileNet(input_shape,n_classes)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 192, 753, 3)]     0         
                                                                 
 conv2d_6 (Conv2D)           (None, 96, 377, 32)       864       
                                                                 
 batch_normalization_10 (Bat  (None, 96, 377, 32)      128       
 chNormalization)                                                
                                                                 
 re_lu_10 (ReLU)             (None, 96, 377, 32)       0         
                                                                 
 depthwise_conv2d_4 (Depthwi  (None, 96, 377, 32)      288       
 seConv2D)                                                       
                                                                 
 batch_normalization_11 (Bat  (None, 96, 377, 32)      128 

In [124]:
display(list(y_train).count(0),
        list(y_train).count(1),
        list(y_train).count(2),
        list(y_train).count(3),
        len(list(y_train)))

70

93

28

28

219

In [179]:
opt=optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, 
epsilon=1e-08, decay=0.0001)

In [180]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
class_weights

array([0.78214286, 0.58870968, 1.95535714, 1.95535714])

In [182]:
from keras import backend as K
import tensorflow_addons as tfa
#from sklearn.metrics import f1_score

coh_kap=tfa.metrics.CohenKappa(num_classes=4, sparse_labels=True)
f1_score=tfa.metrics.F1Score(num_classes=4, threshold=0.5)

In [197]:
sample_weight = np.ones(shape=(len(y_train),))

sample_weight[y_train == 0] = class_weights[0]
sample_weight[y_train == 1] = class_weights[1]
sample_weight[y_train == 2] = class_weights[2]-.5
sample_weight[y_train == 3] = class_weights[3]
display(sample_weight)

array([1.45535714, 0.78214286, 0.78214286, 0.58870968, 0.78214286,
       1.45535714, 0.58870968, 1.45535714, 1.95535714, 0.58870968,
       0.78214286, 0.78214286, 0.78214286, 1.45535714, 0.58870968,
       0.78214286, 0.78214286, 0.78214286, 0.58870968, 0.58870968,
       0.78214286, 0.58870968, 0.58870968, 0.78214286, 1.45535714,
       0.58870968, 0.78214286, 0.78214286, 0.58870968, 0.78214286,
       0.78214286, 0.78214286, 0.78214286, 0.58870968, 1.95535714,
       1.45535714, 0.78214286, 1.95535714, 0.58870968, 0.58870968,
       0.58870968, 1.95535714, 0.58870968, 0.78214286, 0.58870968,
       0.58870968, 0.78214286, 0.78214286, 0.58870968, 0.58870968,
       0.58870968, 0.78214286, 0.78214286, 1.45535714, 0.58870968,
       0.58870968, 0.78214286, 0.58870968, 0.78214286, 1.45535714,
       0.78214286, 0.78214286, 0.78214286, 0.58870968, 0.78214286,
       0.58870968, 1.45535714, 0.58870968, 0.58870968, 1.95535714,
       0.78214286, 0.78214286, 0.78214286, 1.45535714, 0.58870

In [198]:
model.compile(optimizer=opt,loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=[coh_kap], sample_weight_mode="temporal")

In [199]:
md = model.fit(X_train, y_train, epochs=2, validation_data= (X_val, y_val), sample_weight=sample_weight)

Epoch 1/2
7/7 [==============================] - 23s 3s/step - loss: 0.4034 - cohen_kappa: 0.4414 - val_loss: 1.4238 - val_cohen_kappa: 0.0000e+00
Epoch 2/2
7/7 [==============================] - 19s 3s/step - loss: 0.3669 - cohen_kappa: 0.8550 - val_loss: 1.4212 - val_cohen_kappa: 0.0000e+00


In [200]:
yt=model.predict(X_test, verbose=0)
yt

array([[0.21846236, 0.26112014, 0.28438926, 0.23602818],
       [0.22011256, 0.26369864, 0.2791176 , 0.2370712 ],
       [0.22179306, 0.2655544 , 0.27929118, 0.23336141],
       [0.21733013, 0.26383397, 0.28821322, 0.23062265],
       [0.2152655 , 0.26206437, 0.2871233 , 0.23554683],
       [0.21353523, 0.26349702, 0.287631  , 0.2353367 ],
       [0.21612257, 0.26356688, 0.28780246, 0.23250806],
       [0.22546421, 0.2611493 , 0.27855203, 0.23483448],
       [0.21211907, 0.26629013, 0.2836595 , 0.23793124],
       [0.221303  , 0.2573706 , 0.28212   , 0.23920636],
       [0.22383851, 0.26366884, 0.28308904, 0.22940359],
       [0.22417045, 0.26097503, 0.27883977, 0.23601477],
       [0.22051372, 0.26072142, 0.293565  , 0.22519988],
       [0.22572573, 0.26564738, 0.27922434, 0.22940257],
       [0.22240314, 0.26742953, 0.27589035, 0.23427697],
       [0.22528863, 0.2576853 , 0.28072184, 0.23630415],
       [0.21931039, 0.26745543, 0.27982402, 0.23341018],
       [0.2245558 , 0.2624866 ,

In [201]:
for i in range(yt.shape[0]):
    print(str(y_train[i])+"  "+str(np.argmax(yt[i])))

2  2
0  2
0  2
1  2
0  2
2  2
1  2
2  2
3  2
1  2
0  2
0  2
0  2
2  2
1  2
0  2
0  2
0  2
1  2
1  2
0  2
1  2
1  2
0  2
2  2
1  2
0  2
0  2
1  2
0  2
0  2
0  2
0  2
1  2
3  2
2  2
0  2
3  2
1  2
1  2
1  2
3  2
1  2
0  2
1  2
1  2
0  2
0  2
1  2
1  2
1  2
0  2
0  2
2  2
1  2
1  2
0  2
1  2
0  2
2  2
0  2
0  2
0  2
1  2
0  2
1  2
2  2
1  2
1  2
3  2
0  2
0  2
0  2
2  2
1  2
1  2
3  2
3  2
1  2
1  2
0  2
1  2
2  2
3  2
1  2
0  2
1  2
1  2
0  2
1  2
2  2
3  2
0  2
